# Visualising PAG neurons in CCF space
In this notebook we will load the .csv file containing the metadata from our PAG_scRNAseq project and use the CCF coordinates obtained after registration with Sharp-Track to visualise our sequenced cells with Brainrender. We will also write some code to generate some figures for the thesis.

### 1 | Import packages and set defaults

In [ ]:
import brainrender
from brainrender import Scene, Animation
from brainrender.actors import Points
from vedo import settings as vsettings
from brainrender.video import VideoMaker
import pandas as pd # used to load the cwv
import numpy as np # used to set beginning and end of a custom slice
from vedo import embedWindow, Plotter, show  # <- this will be used to render an embedded scene 


# // DEFAULT SETTINGS //
# You can see all the default settings here: https://github.com/brainglobe/brainrender/blob/19c63b97a34336898871d66fb24484e8a55d4fa7/brainrender/settings.py

# --------------------------- brainrender settings --------------------------- #
# Change some of the default settings
brainrender.settings.BACKGROUND_COLOR = "white" # color of the background window (defaults to "white", try "blackboard")
brainrender.settings.DEFAULT_ATLAS = "allen_mouse_25um"  # default atlas
brainrender.settings.DEFAULT_CAMERA = "three_quarters"  # Default camera settings (orientation etc. see brainrender.camera.py)
brainrender.settings.INTERACTIVE = False  # rendering interactive ?
brainrender.settings.LW = 2  # e.g. for silhouettes
brainrender.settings.ROOT_COLOR = [0.4, 0.4, 0.4]  # color of the overall brain model's actor (defaults to [0.8, 0.8, 0.8])
brainrender.settings.ROOT_ALPHA = 0.2  # transparency of the overall brain model's actor (defaults to 0.2)
brainrender.settings.SCREENSHOT_SCALE = 1  # values >1 yield higher resolution screenshots
brainrender.settings.SHADER_STYLE = "cartoon"  # affects the look of rendered brain regions, values can be: ["metallic", "plastic", "shiny", "glossy", "cartoon"] and can be changed in interactive mode
brainrender.settings.SHOW_AXES = False
brainrender.settings.WHOLE_SCREEN = True  # If true render window is full screen
brainrender.settings.OFFSCREEN = False

# ------------------------------- vedo settings ------------------------------ #
# For transparent background with screenshots
vsettings.screenshotTransparentBackground = True  # vedo for transparent bg
vsettings.useFXAA = False  # This needs to be false for transparent bg


# // SET PARAMETERS //
# Save folder
save_folder = r"D:\Dropbox (UCL)\Project_transcriptomics\analysis\PAG_scRNAseq_brainrender\output"

#### 1.1 | Check atlas availability
Brainrender integrates several atlases that can be used to visualise and explore brain anatomy. We can check which atlases are available, take a look at the ones we have already downloaded, and render a basic scene with the axis to get an idea of the overall picture.

In [ ]:
# Run this to get a list of the available atlases:
from bg_atlasapi import show_atlases
show_atlases()

In [ ]:
# Find the dimensions of an atlas:
from bg_atlasapi.bg_atlas import BrainGlobeAtlas
atlas = BrainGlobeAtlas("allen_mouse_25um")
reference_image = atlas.reference
print(reference_image.shape)

Currently, among the atlases we can use for mouse data:
* allen_mouse_10um_v1.2 - Volume dimension of \[AP, SI, LR\] equivalent to \[1320, 800, 1140\]
* allen_mouse_25um_v1.2 - Volume dimension of \[AP, SI, LR\] equivalent to \[528, 320, 456\] (default atlas)
* kim_mouse_10um_v1.0 - Volume dimension of \[AP, SI, LR\] equivalent to \[1320, 800, 1140\]
* kim_unified_25um_v1.0 - Volume dimension of \[AP, SI, LR\] equivalent to \[528, 320, 456\]
* kim_unified_50um_v1.0 - Volume dimension of \[AP, SI, LR\] equivalent to \[264, 160, 228\]
* osten_mouse_10um_v1.1 - Volume dimension of \[AP, SI, LR\] equivalent to \[1320, 800, 1140\]

The CCF coordinates we obtained from Sharp-Track are at 10um resolution, with a Volume dimension of \[AP, SI, LR\] equivalent to \[1320, 800, 1140\]

In [ ]:
embedWindow(None)  # <- this will make your scene popup

# Create a scene with the with the preferred atlas and check the dimensions
brainrender.settings.SHOW_AXES = True
scene = Scene(root = True, atlas_name = 'allen_mouse_25um', inset = False, title = 'allen_mouse_25um', screenshots_folder = save_folder, plotter = None)

scene.render(interactive = True, camera = "sagittal", zoom = 1) # make sure to press 'Esc' to close not 'q' or kernel dies

### 2 | Load metadata including CCF coordinates
Other options to load registered points include `add_cells_from_file` and `get_probe_points_from_sharptrack`.

In [ ]:
# Load data
pag_data = pd.read_csv("D:\\Dropbox (UCL - SWC)\\Project_transcriptomics\\analysis\\PAG_scRNAseq_brainrender\\PAG_scRNAseq_metadata_211018.csv")

# Look at the first 5 rows of the metadata
pag_data.head()

In [ ]:
# List all column names in data
pag_data.columns

#### 2.1 | Scaling up coordinates
The CCF coordinates were obtained by registering images to the Allen Brain Atlas using Sharp-Track (see Shamash et al. bioRxiv 2018 and https://github.com/cortex-lab/allenCCF), which yields coordinates at a resolution of 10 micrometers. In the Allen Brain Atlas, a point at coordinates \[1, 0, 0\] is at 10um from the origin (in other words, 1 unit of the atlas space equals 10um). However, BrainRender's space is at 1um resolution, so the first thing we need to do is to scale up the coordinate values by 10 to get them to match correctly.

In [ ]:
# Inspect the CCF coordinates for each cell
pag_data[["cell.id", "cell.type", "PAG.area", "CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"]]

In [ ]:
# Scale up data
sharptrack_to_brainrender_scale_factor = 10 # Sharp-Track uses a 10um reference atlas so the coordinates need to be scaled to match brainrender's

pag_data["CCF.AllenAP"] *= sharptrack_to_brainrender_scale_factor
pag_data["CCF.AllenDV"] *= sharptrack_to_brainrender_scale_factor
pag_data["CCF.AllenML"] *= sharptrack_to_brainrender_scale_factor

pag_data[["cell.id", "cell.type", "CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"]]

### 3 | Rendering cells with Brainrender
Now that we have the coordinates at the right scale, we can render our cells in Brainrender and colour them according to any metadata we want. We will prepare different renderings in each of the following code chunks.

#### 3.1 | Selecting a subset of cells
We can first take a look at how to use the metadata to select subsets of cells. This will be useful to either render just some of the cells, or to color cells based on some metadata info.

In [ ]:
# We can subset cells using any criteria we want. For instance, let's keep cells coming from each hemisphere in separate variables:
cells_hemisphere_right = pag_data.loc[pag_data["PAG.hemisphere"] == "right"]
cells_hemisphere_left = pag_data.loc[pag_data["PAG.hemisphere"] == "left"]
cells_hemisphere_right.head()

In [ ]:
# We can also use multiple criteria at the same time, such as hemisphere and cell type:
vgat_cells_hemisphere_left = pag_data.loc[(pag_data["PAG.hemisphere"] == "left")&(pag_data["cell.type"] == "VGAT")]
vgat_cells_hemisphere_left.head()

We can now render a scene adding each subset independently, so we can tweak variables such as color or size separately. However, brainrender requires an array of 3 values as coordinates, so we need to get the values out instead of subsetting the dataframe and providing it as input when creating a scene: 

In [ ]:
column_names = ["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"] # name of the columns containing the CCF coordinates
cells_hemisphere_right[column_names].head().values # brainrender needs a numpy array as coordinates. Without the .values we get a dataframe.

Now we can render a scene and color the cells on the right and left hemisphere differently:

In [ ]:
embedWindow(None)  # <- this will make your scene popup
brainrender.settings.SHOW_AXES = False # Set this back to False

# Create a variable containing the XYZ coordinates of the cells.
column_names = ["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"] # name of the columns containing the CCF coordinates

# // CREATE SCENE //
scene = Scene(root = True, atlas_name = 'allen_mouse_25um', inset = False, title = 'Aspirated Cells', screenshots_folder = save_folder, plotter = None)

# // ADD REGIONS AND CELLS//
scene.add_brain_region("PAG", alpha = 0.1, color = "darkgoldenrod", silhouette = None, hemisphere = "both")
scene.add(Points(cells_hemisphere_right[column_names].values, name = "right hemisphere", colors = "salmon", alpha = 1, radius = 20, res = 16))
scene.add(Points(cells_hemisphere_left[column_names].values, name = "left hemisphere", colors = "skyblue", alpha = 1, radius = 20, res = 16))

# // RENDER INTERACTIVELY //
# Render interactively. You can press "s" to take a screenshot
scene.render(interactive = True, camera = "sagittal", zoom = 1) # make sure to press 'Esc' to close not 'q' or kernel dies

### 4 | Rendering VGAT and VGluT2 cells for Chapter 2
We can follow the strategy above and assign each cell type to a different variable and render them separately.

#### 4.1 | Plot brain, PAG, and aspirated cells

We have increased the screenshot resoolution, changed the save folder, and increased the radius of the cells so they are visible. 

We will use "plastic" as shader style in this overview figure, have the axes so we can draw a scale bar, and render using the Top camera and a 1.1x zoom.

Once rendered, save a screenshot by pressing "s".

In [ ]:
embedWindow(None)  # <- this will make your scene popup
brainrender.settings.SHOW_AXES = True
brainrender.settings.SCREENSHOT_SCALE = 10 # values >1 yield higher resolution screenshots
brainrender.settings.SHADER_STYLE = "plastic" # affects the look of rendered brain regions, values can be: ["metallic", "plastic", "shiny", "glossy", "cartoon"] and can be changed in interactive mode
brainrender.settings.ROOT_COLOR = [0.4, 0.4, 0.4] # color of the overall brain model's actor (defaults to [0.8, 0.8, 0.8])
brainrender.settings.ROOT_ALPHA = 0.2 # transparency of the overall brain model's actor (defaults to 0.2)
save_folder_thesis = r"D:\Dropbox (UCL)\Project_transcriptomics\analysis\PAG_scRNAseq_brainrender\output\figures_thesis_chapter_2"

# Create a variable containing the XYZ coordinates of the cells.
column_names = ["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"] # name of the columns containing the CCF coordinates

# Color cells according to whether they are excitatory or inhibitory:
vgat_cells = pag_data.loc[pag_data["cell.type"] == "VGAT"]
vglut2_cells =  pag_data.loc[pag_data["cell.type"] == "VGluT2"]

# // CREATE SCENE //
scene = Scene(root = True, atlas_name = 'allen_mouse_25um', inset = False, title = None, screenshots_folder = save_folder_thesis, plotter = None)

# // ADD REGIONS AND CELLS//
scene.add_brain_region("PAG", alpha = 0.2, color = "darkgoldenrod", silhouette = None, hemisphere = "both")
#scene.add_brain_region("SCm", alpha = 0.1, color = "olivedrab", silhouette = None, hemisphere = "both")
scene.add(Points(vgat_cells[column_names].values, name = "vgat", colors = "#FF8080", alpha = 1, radius = 30, res = 16)) # colour is #FF8080 in figures, but salmon is the same
scene.add(Points(vglut2_cells[column_names].values, name = "vglut2", colors = "#0F99B2", alpha = 1, radius = 30, res = 16)) # colour is #0F99B2 in figures, but skyblue looks good here

# // RENDER INTERACTIVELY //
# Render interactively. You can press "s" to take a screenshot
scene.render(interactive = True, camera = "top", zoom = 1.1) # cameras can be "sagittal", "sagittal2", "frontal", "top", "top_side", "three_quarters"

#### 4.2 | Plot PAG and aspirated cells at different angles

Now we want to render and save some images in which we zoom into the PAG to visualise the registered cells. We can color by cell type or by PAG subdivision.

Colouring cells by cell_type (VGAT or VGluT2), we will save the following screenshots:
* Axes True, Zoom 4, Top Camera
* Axes False, Zoom 4, Top Camera
* Axes False, Zoom 4, Sagittal Camera
* Axes False, Zoom 4, Three quarters Camera

In [ ]:
embedWindow(None)  # <- this will make your scene popup
brainrender.settings.SHOW_AXES = False
brainrender.settings.SCREENSHOT_SCALE = 10 # values >1 yield higher resolution screenshots
brainrender.settings.SHADER_STYLE = "cartoon" # affects the look of rendered brain regions, values can be: ["metallic", "plastic", "shiny", "glossy", "cartoon"] and can be changed in interactive mode
brainrender.settings.ROOT_COLOR = [0.4, 0.4, 0.4] # color of the overall brain model's actor (defaults to [0.8, 0.8, 0.8])
brainrender.settings.ROOT_ALPHA = 0.2 # transparency of the overall brain model's actor (defaults to 0.2)
save_folder_thesis = r"D:\Dropbox (UCL)\Project_transcriptomics\analysis\PAG_scRNAseq_brainrender\output\figures_thesis_chapter_2"

# Create a variable containing the XYZ coordinates of the cells.
column_names = ["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"] # name of the columns containing the CCF coordinates

# Color cells according to whether they are excitatory or inhibitory:
vgat_cells = pag_data.loc[pag_data["cell.type"] == "VGAT"]
vglut2_cells =  pag_data.loc[pag_data["cell.type"] == "VGluT2"]

# // CREATE SCENE //
scene = Scene(root = False, atlas_name = 'allen_mouse_25um', inset = False, title = None, screenshots_folder = save_folder_thesis, plotter = None)

# // ADD REGIONS AND CELLS//
scene.add_brain_region("PAG", alpha = 0.2, color = "darkgoldenrod", silhouette = None, hemisphere = "both")
#scene.add_brain_region("SCm", alpha = 0.1, color = "olivedrab", silhouette = None, hemisphere = "both")
scene.add(Points(vgat_cells[column_names].values, name = "vgat", colors = "#FF8080", alpha = 1, radius = 20, res = 16)) # colour is #FF8080 in figures, but salmon is the same
scene.add(Points(vglut2_cells[column_names].values, name = "vglut2", colors = "#0F99B2", alpha = 1, radius = 20, res = 16)) # colour is #0F99B2 in figures, but skyblue looks good here

# // RENDER INTERACTIVELY //
# Render interactively. You can press "s" to take a screenshot
scene.render(interactive = True, camera = "three_quarters", zoom = 4) # cameras can be "sagittal", "sagittal2", "frontal", "top", "top_side", "three_quarters"

Colouring cells by cell_type (VGAT or VGluT2), we will save the following screenshots:
* Axes False, Zoom 4, Frontal Camera, sliced

In [ ]:
embedWindow(None)  # <- this will make your scene popup
brainrender.settings.SHOW_AXES = False
brainrender.settings.SCREENSHOT_SCALE = 10 # values >1 yield higher resolution screenshots
brainrender.settings.SHADER_STYLE = "cartoon" # affects the look of rendered brain regions, values can be: ["metallic", "plastic", "shiny", "glossy", "cartoon"] and can be changed in interactive mode
brainrender.settings.ROOT_COLOR = [0.4, 0.4, 0.4] # color of the overall brain model's actor (defaults to [0.8, 0.8, 0.8])
brainrender.settings.ROOT_ALPHA = 0.2 # transparency of the overall brain model's actor (defaults to 0.2)
save_folder_thesis = r"D:\Dropbox (UCL)\Project_transcriptomics\analysis\PAG_scRNAseq_brainrender\output\figures_thesis_chapter_2"

# Create a variable containing the XYZ coordinates of the cells.
column_names = ["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"] # name of the columns containing the CCF coordinates

# Color cells according to whether they are excitatory or inhibitory:
vgat_cells = pag_data.loc[pag_data["cell.type"] == "VGAT"]
vglut2_cells =  pag_data.loc[pag_data["cell.type"] == "VGluT2"]

# // CREATE SCENE //
scene = Scene(root = False, atlas_name = 'allen_mouse_25um', inset = False, title = None, screenshots_folder = save_folder_thesis, plotter = None)

# // ADD REGIONS AND CELLS//
pag = scene.add_brain_region("PAG", alpha = 0.2, color = "darkgoldenrod", silhouette = None, hemisphere = "both")
#scene.add_brain_region("SCm", alpha = 0.1, color = "olivedrab", silhouette = None, hemisphere = "both")
scene.add(Points(vgat_cells[column_names].values, name = "vgat", colors = "#FF8080", alpha = 1, radius = 20, res = 16)) # colour is #FF8080 in figures, but salmon is the same
scene.add(Points(vglut2_cells[column_names].values, name = "vglut2", colors = "#0F99B2", alpha = 1, radius = 20, res = 16)) # colour is #0F99B2 in figures, but skyblue looks good here

# // SLICE SCENE //
slice_start = pag.centerOfMass() + np.array([-150, 0, 0]) # X microns from center of mass towards the nose (if positive) or cerebellum (if negative)
slice_end = pag.centerOfMass() + np.array([+800, 0, 0]) # X microns from center of mass towards the nose (if adding) or cerebellum (if subtracting)

for p, n in zip((slice_start, slice_end), (1, -1)):
    plane = scene.atlas.get_plane(pos = p, norm = (n, 0, 0))
    scene.slice(plane, actors = pag, close_actors = True)

# // RENDER INTERACTIVELY //
# Render interactively. You can press "s" to take a screenshot
scene.render(interactive = True, camera = "frontal", zoom = 4) # cameras can be "sagittal", "sagittal2", "frontal", "top", "top_side", "three_quarters"

Colouring cells by PAG subdivision, we will save the following screenshots:
* Axes False, Zoom 4, Top Camera
* Axes False, Zoom 4, Sagittal Camera
* Axes False, Zoom 4, Three quarters Camera

In [ ]:
embedWindow(None)  # <- this will make your scene popup
brainrender.settings.SHOW_AXES = False # Set this back to False
brainrender.settings.SCREENSHOT_SCALE = 10 # values >1 yield higher resolution screenshots
brainrender.settings.SHADER_STYLE = "cartoon" # affects the look of rendered brain regions, values can be: ["metallic", "plastic", "shiny", "glossy", "cartoon"] and can be changed in interactive mode
brainrender.settings.ROOT_COLOR = [0.4, 0.4, 0.4] # color of the overall brain model's actor (defaults to [0.8, 0.8, 0.8])
brainrender.settings.ROOT_ALPHA = 0.2 # transparency of the overall brain model's actor (defaults to 0.2)
save_folder_thesis = r"D:\Dropbox (UCL)\Project_transcriptomics\analysis\PAG_scRNAseq_brainrender\output\figures_thesis_chapter_2"

# Create a variable containing the XYZ coordinates of the cells.
column_names = ["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"] # name of the columns containing the CCF coordinates

# Color cells according to their subdivision:
dmpag_cells = pag_data.loc[pag_data["PAG.area"] == "dmpag"]
dlpag_cells = pag_data.loc[pag_data["PAG.area"] == "dlpag"]
lpag_cells = pag_data.loc[pag_data["PAG.area"] == "lpag"]
vlpag_cells = pag_data.loc[pag_data["PAG.area"] == "vlpag"]

# // CREATE SCENE //
scene = Scene(root = False, atlas_name = 'allen_mouse_25um', inset = False, title = None, screenshots_folder = save_folder_thesis, plotter = None)

# // ADD REGIONS AND CELLS//
scene.add_brain_region("PAG", alpha = 0.2, color = "darkgoldenrod", silhouette = None, hemisphere = "both")
#scene.add_brain_region("SCm", alpha = 0.1, color = "olivedrab", silhouette = None, hemisphere = "both")
scene.add(Points(dmpag_cells[column_names].values, name = "dmpag", colors = "cornflowerblue", alpha = 1, radius = 20, res = 16))
scene.add(Points(dlpag_cells[column_names].values, name = "dlpag", colors = "darkorange", alpha = 1, radius = 20, res = 16))
scene.add(Points(lpag_cells[column_names].values, name = "lpag", colors = "forestgreen", alpha = 1, radius = 20, res = 16))
scene.add(Points(vlpag_cells[column_names].values, name = "vlpag", colors = "firebrick", alpha = 1, radius = 20, res = 16))

# // RENDER INTERACTIVELY //
# Render interactively. You can press "s" to take a screenshot
scene.render(interactive = True, camera = "three_quarters", zoom = 4) # choose one of the cameras: sagittal, sagittal2, frontal, top, top_side, three_quarters

Colouring cells by PAG subdivision, we will save the following screenshots:
* Axes False, Zoom 4, Frontal Camera, sliced

In [ ]:
embedWindow(None)  # <- this will make your scene popup
brainrender.settings.SHOW_AXES = False # Set this back to False
brainrender.settings.SCREENSHOT_SCALE = 10 # values >1 yield higher resolution screenshots
brainrender.settings.SHADER_STYLE = "cartoon" # affects the look of rendered brain regions, values can be: ["metallic", "plastic", "shiny", "glossy", "cartoon"] and can be changed in interactive mode
brainrender.settings.ROOT_COLOR = [0.4, 0.4, 0.4] # color of the overall brain model's actor (defaults to [0.8, 0.8, 0.8])
brainrender.settings.ROOT_ALPHA = 0.2 # transparency of the overall brain model's actor (defaults to 0.2)
save_folder_thesis = r"D:\Dropbox (UCL)\Project_transcriptomics\analysis\PAG_scRNAseq_brainrender\output\figures_thesis_chapter_2"

# Create a variable containing the XYZ coordinates of the cells.
column_names = ["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"] # name of the columns containing the CCF coordinates

# Color cells according to their subdivision:
dmpag_cells = pag_data.loc[pag_data["PAG.area"] == "dmpag"]
dlpag_cells = pag_data.loc[pag_data["PAG.area"] == "dlpag"]
lpag_cells = pag_data.loc[pag_data["PAG.area"] == "lpag"]
vlpag_cells = pag_data.loc[pag_data["PAG.area"] == "vlpag"]

# // CREATE SCENE //
scene = Scene(root = False, atlas_name = 'allen_mouse_25um', inset = False, title = None, screenshots_folder = save_folder_thesis, plotter = None)

# // ADD REGIONS AND CELLS//
pag = scene.add_brain_region("PAG", alpha = 0.2, color = "darkgoldenrod", silhouette = None, hemisphere = "both")
#scene.add_brain_region("SCm", alpha = 0.1, color = "olivedrab", silhouette = None, hemisphere = "both")
scene.add(Points(dmpag_cells[column_names].values, name = "dmpag", colors = "cornflowerblue", alpha = 1, radius = 20, res = 16))
scene.add(Points(dlpag_cells[column_names].values, name = "dlpag", colors = "darkorange", alpha = 1, radius = 20, res = 16))
scene.add(Points(lpag_cells[column_names].values, name = "lpag", colors = "forestgreen", alpha = 1, radius = 20, res = 16))
scene.add(Points(vlpag_cells[column_names].values, name = "vlpag", colors = "firebrick", alpha = 1, radius = 20, res = 16))

# // SLICE SCENE //
slice_start = pag.centerOfMass() + np.array([-150, 0, 0]) # X microns from center of mass towards the nose (if positive) or cerebellum (if negative)
slice_end = pag.centerOfMass() + np.array([+800, 0, 0]) # X microns from center of mass towards the nose (if adding) or cerebellum (if subtracting)

for p, n in zip((slice_start, slice_end), (1, -1)):
    plane = scene.atlas.get_plane(pos = p, norm = (n, 0, 0))
    scene.slice(plane, actors = pag, close_actors = True)

# // RENDER INTERACTIVELY //
# Render interactively. You can press "s" to take a screenshot
scene.render(interactive = True, camera = "frontal", zoom = 4) # choose one of the cameras: sagittal, sagittal2, frontal, top, top_side, three_quarters